In [7]:
import os
import os.path
from collections import Counter

In [8]:
# сontera_runner
scaf_test = "/mnt/project/zilov/sqeezemeta/test_run_staka2/Sample1/results/01.Sample1.fasta"
out_dir = "/mnt/project/zilov/sqeezemeta/test_run_staka2/Sample1/results/contera/"
outfmt = "/mnt/project/zilov/sqeezemeta/test_run_staka2/Sample1/results/contera/staka2.outfmt6"
adapters = os.path.abspath("./adapters_db/adaptor_fasta.fna")
contera = os.path.abspath("./contera.py")

command = f"{contera} -s {scaf_test} -b {outfmt} -a {adapters} -o {out_dir}"
print(command)

/home/dzilov/scripts/contera/contera.py -s /mnt/project/zilov/sqeezemeta/test_run_staka2/Sample1/results/01.Sample1.fasta -b /mnt/project/zilov/sqeezemeta/test_run_staka2/Sample1/results/contera/staka2.outfmt6 -a /home/dzilov/scripts/contera/adapters_db/adaptor_fasta.fna -o /mnt/project/zilov/sqeezemeta/test_run_staka2/Sample1/results/contera/


'/mnt/data/ncbi/refseq/bacteria/fna'

In [3]:
#input
blast_db = "/mnt/data/ncbi/blast/db/nt"
scaffold = "/home/dzilov/scripts/contera/test_data/scaffolds.fasta"
adapters_db = "adapters_db/adaptor_fasta.fna"
threads = 50

blastn_taxid_out = "/home/dzilov/scripts/contera/results/Ecoli19.outfmt6"
output = "/home/dzilov/scripts/contera/results/"
scaforarg = scaffold.split("/")[-1].split(".")[0]
prefix = blastn_taxid_out.split("/")[-1].split(".")[0]

scaffolds_filtered = output + scaforarg + "_filtered.fasta"
contera_report = output + prefix + ".contera_report.txt"
vectors_log = output + prefix + ".vectors.tsv"
phages_log = output + prefix + ".phages.tsv"
adapters_report = output + prefix + ".adapters_report.outfmt6"

In [186]:
os.path.abspath("./results/")

'/home/dzilov/scripts/contera/results'

In [167]:
#нужно получить из бласта:
# 1) Процентное отношение бактерий в сборке (каунтер по sscinames)
# 2) Три самых часто встречающихся айди таксона (каунтер по staxids)
# 3) Части контигов, которые нужно удалить (от qstart до qend в контиге qseqid)

# итог работы контеры
# 1) Таксон бактерии
# 2) Контаминации в контигах (или гор. перенос)
# 3) Адаптеры - проверка на наличие, если есть - удаление
# 4) Референсный геном и аннотация для дальнейших действий

In [5]:
# В RULES/CLEANING.SF BLAST, НЕ ПИХАТЬ В КОНЕЧНЫЙ СКРИПТ

scaf_test = "/mnt/project/zilov/sqeezemeta/test_run_staka2/Sample1/results/01.Sample1.fasta"
out_file = "/mnt/project/zilov/sqeezemeta/test_run_staka2/Sample1/results/contera/staka2.outfmt6"

command = """blastn \
-db nt \
-query %s \
-evalue 0.00001 \
-outfmt '6 qseqid sallacc pident qstart qend length evalue sscinames staxids' \
-max_target_seqs 5 \
-num_threads %s \
-out %s""" % (scaf_test, threads, out_file)

print(command)
#os.system(command)

# ФАЙЛ БЛАСТА ИДЕТ ДАЛЬШЕ КАК ИНПУТ В КОНТЕРУ

blastn -db nt -query /mnt/project/zilov/sqeezemeta/test_run_staka2/Sample1/results/01.Sample1.fasta -evalue 0.00001 -outfmt '6 qseqid sallacc pident qstart qend length evalue sscinames staxids' -max_target_seqs 5 -num_threads 50 -out /mnt/project/zilov/sqeezemeta/test_run_staka2/Sample1/results/contera/staka2.outfmt6


In [3]:
# ПЕРВАЯ СТАДИЯ, ЧИТАЕМ БЛАСТ, НАХОДИМ ДЛИНУ УЧАСТКОВ ПО БАКТЕРИЯМ И ТАКСОНАМ

bact_taxid = {}
contigs = {} # ЗАПИСАВАЕМ КОНТИГИ В СЛОВАРЬ, ДАЛЬШЕ ЕГО ИСПОЛЬЗУЕМ ДЛЯ ПОИСКА КОНТАМИНАЦИЙ

vectors = []
phages = []

bact_length = Counter()
taxid_length = Counter()
total_length = 0

bact_seq = {}
with open(blastn_taxid_out) as fh:
    for line in fh:
        line = line.strip().split("\t")
        print(line)
        #Column names
        contig = line[0]
        contigs[contig] = []
        ident = float(line[2])
        start = line[3]
        end = line[4]
        length = int(line[5])
        bact_full_name = line[7]
        bacteria = " ".join(line[7].split(" ")[:2])
        taxid = line[8]
        
        if "vector" in bact_full_name or "Vector" in bact_full_name:
            vectors.append(line)
        if "phage" in bact_full_name or "Phage" in bact_full_name:
            phages.append(line)
        
        bact_length[bacteria] += length
        taxid_length[taxid] += length
        total_length += length
        
        bact_seq[bacteria] = contig + ":" + start + ".." + end

['NODE_1_length_344957_cov_96.369759', 'gi|1609454293|gb|CP038505.1|', '99.973', '859', '182709', '181860', '0.0', 'Escherichia coli', '562']
['NODE_1_length_344957_cov_96.369759', 'gi|1609454293|gb|CP038505.1|', '99.995', '185883', '344951', '159069', '0.0', 'Escherichia coli', '562']
['NODE_1_length_344957_cov_96.369759', 'gi|1609454293|gb|CP038505.1|', '99.968', '182810', '185903', '3094', '0.0', 'Escherichia coli', '562']
['NODE_1_length_344957_cov_96.369759', 'gi|1609454293|gb|CP038505.1|', '97.778', '2609', '3733', '1125', '0.0', 'Escherichia coli', '562']
['NODE_1_length_344957_cov_96.369759', 'gi|1609454293|gb|CP038505.1|', '95.843', '1', '866', '866', '0.0', 'Escherichia coli', '562']
['NODE_1_length_344957_cov_96.369759', 'gi|1609454293|gb|CP038505.1|', '95.833', '1', '864', '864', '0.0', 'Escherichia coli', '562']
['NODE_1_length_344957_cov_96.369759', 'gi|1609454293|gb|CP038505.1|', '95.481', '1', '863', '863', '0.0', 'Escherichia coli', '562']
['NODE_1_length_344957_cov_96

['NODE_25_length_67962_cov_137.190713', 'gi|1726040740|gb|CP042899.1|', '99.219', '1', '128', '128', '2.60e-54', 'Escherichia coli', '562']
['NODE_25_length_67962_cov_137.190713', 'gi|1726040740|gb|CP042899.1|', '99.219', '1', '128', '128', '2.60e-54', 'Escherichia coli', '562']
['NODE_25_length_67962_cov_137.190713', 'gi|1726040740|gb|CP042899.1|', '100.000', '67839', '67962', '124', '9.35e-54', 'Escherichia coli', '562']
['NODE_25_length_67962_cov_137.190713', 'gi|1726040740|gb|CP042899.1|', '100.000', '1', '124', '124', '9.35e-54', 'Escherichia coli', '562']
['NODE_25_length_67962_cov_137.190713', 'gi|1726040740|gb|CP042899.1|', '99.213', '67836', '67962', '127', '9.35e-54', 'Escherichia coli', '562']
['NODE_25_length_67962_cov_137.190713', 'gi|1726040740|gb|CP042899.1|', '98.438', '1', '128', '128', '1.21e-52', 'Escherichia coli', '562']
['NODE_25_length_67962_cov_137.190713', 'gi|1726040740|gb|CP042899.1|', '98.438', '1', '128', '128', '1.21e-52', 'Escherichia coli', '562']
['NODE

['NODE_104_length_1933_cov_51.046512', 'gi|1726040740|gb|CP042899.1|', '82.197', '1232', '1493', '264', '1.22e-53', 'Escherichia coli', '562']
['NODE_104_length_1933_cov_51.046512', 'gi|1726040740|gb|CP042899.1|', '89.385', '1757', '1933', '179', '4.37e-53', 'Escherichia coli', '562']
['NODE_104_length_1933_cov_51.046512', 'gi|1726040740|gb|CP042899.1|', '94.245', '6', '143', '139', '9.47e-50', 'Escherichia coli', '562']
['NODE_104_length_1933_cov_51.046512', 'gi|1726040740|gb|CP042899.1|', '80.833', '1232', '1470', '240', '1.60e-42', 'Escherichia coli', '562']
['NODE_104_length_1933_cov_51.046512', 'gi|1706544408|gb|CP041532.1|', '100.000', '1', '1933', '1933', '0.0', 'Escherichia coli', '562']
['NODE_104_length_1933_cov_51.046512', 'gi|1706544408|gb|CP041532.1|', '92.647', '1216', '1486', '272', '1.13e-103', 'Escherichia coli', '562']
['NODE_104_length_1933_cov_51.046512', 'gi|1706544408|gb|CP041532.1|', '92.279', '1216', '1486', '272', '5.27e-102', 'Escherichia coli', '562']
['NODE_

In [170]:
# ШАГ 2. НАХОДИМ НАЗВАНИЕ НАШЕЙ БАКТЕРИИ И ЕЕ ТАКСОН 

target_bact_seq = {}
bact_perc = Counter()
for bact, length in bact_length.most_common():
    bact_perc[bact] = length/total_length * 100
    
target_bact_sciname = bact_perc.most_common(1)[0][0]
    
print("Bacterial content is:", bact_perc.most_common(5))
print("Most common taxon ID is:", taxid_length.most_common(5))

# ЗАПИСЫВАЕМ В ФАЙЛ


bacteria_id = []
for i in taxid_length.most_common()[:3]:
    bacteria_id.append(str(i[0]) + " = " + str(i[1]/total_length * 100) + "%")

Bacterial content is: [('Escherichia coli', 98.64001261692864), ('Salmonella enterica', 0.4629528003266352), ('Shigella sonnei', 0.31416017217919956), ('uncultured bacterium', 0.16100092499377766), ('Shigella boydii', 0.10910131062736461)]
Most common taxon ID is: [('562', 28597003), ('696406', 209852), ('624', 93026), ('59201', 88522), ('216592', 81464)]


In [171]:
# ШАГ 3. ЗАПИСЫВАЕМ КОНТАМИНАЦИИ В СЛОВАРЬ

for node, position in contigs.items():
    with open(blastn_taxid_out) as fh:
        for line in fh:
            line = line.strip().split("\t")
            
            #column names
            contig = line[0]
            ident = float(line[2])
            start = line[3]
            end = line[4]
            bacteria = " ".join(line[7].split(" ")[:2])

            if contig == node and bacteria != target_bact_sciname and ident > 80:
                if start + ":" + end + ":" + bacteria in position:
                    continue
                else:
                    position.append(start + ":" + end + ":" + bacteria)
                break
        fh.close()

In [172]:
#ФИЛЬТРУЕМ НАЙДЕННЫЕ КОНТАМИНАЦИИ, ВЛОЖЕННЫЕ УДЛИННЯЕМ
delete_seq = {}

for node, position in contigs.items():
    if position:
        position.sort(key=lambda x : (int(x.split(":")[0]), -1 * int(x.split(":")[1])))
        delete_seq[node] = []
        
        i = 1
        ps,pe = map(int, position[0].split(":")[:2])
        bac = position[i-1].split(":")[2]
        while i < len(position):
            bac = position[i-1].split(":")[2]
            s,e = map(int, position[i].split(":")[:2])
            if pe + 1 < s:
                delete_seq[node].append((ps, pe, bac))
                ps, pe = s, e
                i += 1
                continue
            
            pe = max(e, pe)
            i += 1
        delete_seq[node].append((ps, pe, bac))

In [173]:
# ШАГ 4, УДАЛЯЕМ АДАПТЕРЫ И ПОСЛЕ ЭТОГО КОНТИГИ ДЛИНОЙ МЕНЬШЕ 200 БП

# БЛАСТИМ ГЕНОМ НА АДАПТЕРЫ

command = """
blastn -query %s \
                    -db %s \
                    -outfmt 6 \
                    -max_target_seqs 1000 \
                    -task blastn \
                    -reward 1 \
                    -penalty \
                    -5 \
                    -gapopen 3 \
                    -gapextend 3 \
                    -dust yes \
                    -soft_masking true \
                    -evalue 0.00001 \
                    -searchsp 1750000000000 \
                    -out %s
          """ % (scaffold, adapters_db, adapters_report)

print(command)
os.system(command)


blastn -query /home/dzilov/scripts/contera/test_data/scaffolds.fasta                     -db adapters_db/adaptor_fasta.fna                     -outfmt 6                     -max_target_seqs 1000                     -task blastn                     -reward 1                     -penalty                     -5                     -gapopen 3                     -gapextend 3                     -dust yes                     -soft_masking true                     -evalue 0.00001                     -searchsp 1750000000000                     -out /home/dzilov/scripts/contera/results/Ecoli19.adapters_report.outfmt6
          


0

In [184]:
adapters_to_del = {}
with open(adapters_report) as fh:
    for line in fh:
        line = line.strip().split()
        ids = line[1].split(":")[1]
        contig = line[0]
        start = line[6]
        stop = line[7]
        adapters_to_del[contig] = start + ":" + stop       

In [175]:
delete_seq

{'NODE_24_length_83879_cov_151.567616': [(8822, 77701, 'Salmonella enterica')],
 'NODE_28_length_57865_cov_145.685181': [(36430,
   51308,
   'Salmonella enterica')],
 'NODE_42_length_33347_cov_94.659211': [(1, 25898, 'Shigella boydii')],
 'NODE_60_length_10022_cov_102.927236': [(127, 10022, 'Shigella flexneri')],
 'NODE_62_length_8413_cov_159.049481': [(1085, 7163, 'Salmonella enterica')],
 'NODE_70_length_6173_cov_83.144393': [(127, 6173, 'Klebsiella pneumoniae')],
 'NODE_72_length_5579_cov_89.099230': [(1, 5579, 'Klebsiella pneumoniae')],
 'NODE_81_length_4218_cov_650.290638': [(1895, 4218, 'Salmonella enterica')],
 'NODE_84_length_3906_cov_96.758137': [(80, 3906, 'uncultured bacterium')],
 'NODE_85_length_3820_cov_224.504197': [(1, 3820, 'Shigella flexneri')],
 'NODE_94_length_2849_cov_109.915871': [(1, 2849, 'Enterobacter hormaechei')],
 'NODE_98_length_2440_cov_76.616515': [(1, 2440, 'Enterobacter hormaechei')],
 'NODE_102_length_2199_cov_127.539575': [(122, 2199, 'Shigella flexn

In [112]:
def fasta_reader(fasta_file):
    fasta = {}
    header = None
    with open(fasta_file) as fh:
        for i, line in enumerate(fh):
            line = line.strip()
            if line.startswith(">"):
                if header:
                    fasta[header]="".join(seq)
                header = line
                seq = []
            else:
                seq.append(line)
    if header:
        fasta[header] = "".join(seq)
    return fasta

In [177]:
for contig, position in adapters_to_del.items():
    start = int(position.split(":")[0])-1
    stop = int(position.split(":")[1])
    for key, value in fasta.items():
        if key.startswith(">" + contig):
            line_to_del = value[start:stop]
            value = value.replace(line_to_del, "")
            fasta[key] = value

In [178]:
short_contig = []
for key, value in fasta.items():
    if len(value) < 200:
        short_contig.append(key)

for i in short_contig:
    if i in fasta.keys():
        fasta.pop(i)

In [179]:
with open(scaffolds_filtered, "w") as fw:
    for key, value in fasta.items():
        fw.write(key + "\n" + value + "\n")
fw.close()

In [180]:
with open(contera_report, 'w') as fw:
    fw.write("BACTERIAL CONTENT OF ASSEMBLY IS:\n")
    for i in bact_perc.most_common(5):
        bact = str(i[0])
        perc = str(i[1])
        fw.write(bact + " - " + perc +"%\n")
    
    fw.write("\nTHE MOST COMMON ID IS :\n" + ", ".join(bacteria_id) + "\n\n")
    
    fw.write("FOREIGN BACTERIA SITES:\n")
    for node, position in delete_seq.items():
        fw.write(node + "\t")
        for i in position:
            start_to_stop = ":".join(str(number) for number in i[:2])
            bacteria = i[2]
            fw.write(start_to_stop + "\t" + bacteria + "\n")
    
    fw.write("\n" + str(len(adapters_to_del)) + " ADAPTERS WERE REMOVED IN:\n")
    for node, position in adapters_to_del.items():
        fw.write(node + "\t" + position + "\n")
        
    fw.close()

In [181]:
# ШАГ 5, КАЧАЕМ РЕФЕРЕНСНЫЙ ГЕНОМ И АННОТАЦИЮ

In [182]:
bact_perc.most_common(10)


[('Escherichia coli', 98.64001261692864),
 ('Salmonella enterica', 0.4629528003266352),
 ('Shigella sonnei', 0.31416017217919956),
 ('uncultured bacterium', 0.16100092499377766),
 ('Shigella boydii', 0.10910131062736461),
 ('Klebsiella pneumoniae', 0.09881121920528885),
 ('Shigella flexneri', 0.07869707922830056),
 ('Enterobacter hormaechei', 0.06539459478079268),
 ('Enterobacter cloacae', 0.02599708689436801),
 ('Escherichia albertii', 0.018712634253272683)]

In [183]:
i = (12, 212)
p = ":".join(str(number) for number in i)
print(p)

12:212


In [1]:
## новый счетчик blast
from collections import defaultdict
contig2bactlength = defaultdict(list)

blast_file = "/mnt/projects/zilov/anna_bact/prokaryotic_pipeline/data/bifid7/genome_assembly/QC/blast/Bbif7_S5_L001_.outfmt6"
n = 0
with open(blast_file) as fh:
    for line in fh:
        n += 1
        line = line.strip().split("\t")
        contig = line[0]
        contig2bactlength[contig].append(line)

In [ ]:
#looking for the longest blasted sequence in contig
bact_percentage_in_contig = defaultdict(dict)
for contig_blast in contig2bactlength:
    value = contig2bactlength[contig_blast]
    value.sort(key=lambda x:int(x[5]), reverse=True)
    bact_seq = defaultdict(list)
    i = 0
    for blast_match in value:
        start_stop = [int(blast_match[3]), int(blast_match[4])]
        bacteria = " ".join(blast_match[-2].split()[0:2])
        if not bact_seq[bacteria]:
            bact_seq[bacteria].append(start_stop)
            print(f"written first {bacteria}")
        else:
            print(f"whatchin inside of {bacteria}")
            print(f"wanna put inside {start_stop}")
            i = 1
            for seq_in in bact_seq[bacteria]:
                print(f"will compare it to {seq_in}")
                start_in = seq_in[0]
                stop_in = seq_in[1]
                start_out = start_stop[0]
                stop_out = start_stop[1]
                print(bact_seq)
                if start_out <= start_in and stop_out >= stop_in:
                    seq_in = start_stop
                elif (start_in <= start_out < stop_in) and stop_out > stop_in:
                    seq_in[1] = stop_out
                elif start_out < start_in and (start_in < stop_out <= stop_in):
                    seq_in[0] = start_out
                elif start_out < start_in and stop_out < start_in:
                    if i == len(bact_seq[bacteria]):
                        bact_seq[bacteria].append(start_stop)
                    else:
                        i+=1
                        continue
                elif start_out > stop_in and stop_out > stop_in:
                    if i == len(bact_seq[bacteria]):
                        bact_seq[bacteria].append(start_stop)
                    else:
                        i+=1
                        continue
                elif start_out == start_in and stop_out == stop_in:
                    continue
                elif start_out > start_in and stop_out < stop_in:
                    continue
    bact_percentage_in_contig[contig_blast] = bact_seq

In [81]:
bact2seqparts = defaultdict(dict)
for contig in bact_percentage_in_contig:
    bacteria2len = bact_percentage_in_contig[contig]
    bact_seq = defaultdict(list)
    for bacteria in bacteria2len:
        lengths = bacteria2len[bacteria]
        lengths.sort(key=lambda x:x[0], reverse=False)
        for blast_match in lengths:
            start_stop = [blast_match[0], blast_match[1]]
            if not bact_seq[bacteria]:
                bact_seq[bacteria].append(start_stop)
                print(f"written first {bacteria} to {contig}")
            else:
                #print(f"whatchin inside of {bacteria}")
                #print(f"wanna put inside {start_stop}")
                i = 1
                for seq_in in bact_seq[bacteria]:
                    #print(f"will compare it to {seq_in}")
                    start_in = seq_in[0]
                    stop_in = seq_in[1]
                    start_out = start_stop[0]
                    stop_out = start_stop[1]
                    #print(bact_seq)
                    if start_out <= start_in and stop_out >= stop_in:
                        seq_in = start_stop
                    elif (start_in <= start_out < stop_in) and stop_out > stop_in:
                        seq_in[1] = stop_out
                    elif start_out < start_in and (start_in < stop_out <= stop_in):
                        seq_in[0] = start_out
                    elif start_out < start_in and stop_out < start_in:
                        if i == len(bact_seq[bacteria]):
                            bact_seq[bacteria].append(start_stop)
                        else:
                            i+=1
                            continue
                    elif start_out > stop_in and stop_out > stop_in:
                        if i == len(bact_seq[bacteria]):
                            bact_seq[bacteria].append(start_stop)
                        else:
                            i+=1
                            continue
                    elif start_out == start_in and stop_out == stop_in:
                        continue
                    elif start_out > start_in and stop_out < stop_in:
                        continue
    bact2seqparts[contig] = bact_seq

written first Bifidobacterium adolescentis to NODE_1_length_716629_cov_9.319197
defaultdict(<class 'list'>, {'Bifidobacterium adolescentis': [[50, 264714], [272200, 716629]]})
written first Bifidobacterium adolescentis to NODE_2_length_341399_cov_7.303676
defaultdict(<class 'list'>, {'Bifidobacterium adolescentis': [[1, 214907], [215172, 220822], [220857, 226337], [226535, 226823], [227002, 266834], [267036, 279541], [280536, 323334], [323504, 341399]]})
written first Bifidobacterium bifidum to NODE_3_length_159041_cov_54.907126
defaultdict(<class 'list'>, {'Bifidobacterium bifidum': [[1, 57890], [57991, 122631], [122732, 128062], [128163, 136922], [137023, 148969], [149070, 159041]]})
written first Bifidobacterium adolescentis to NODE_4_length_157392_cov_10.870893
defaultdict(<class 'list'>, {'Bifidobacterium adolescentis': [[1, 19773], [21816, 62028], [62217, 112868], [113433, 130335], [130336, 157392]]})
written first Bifidobacterium bifidum to NODE_5_length_150296_cov_63.517637
def

In [86]:
contig_content = defaultdict(dict)
for contig in bact2seqparts:
    bac_content = defaultdict(list)
    bacterias = bact2seqparts[contig]
    contig_length = int(contig.split("_")[3])
    for bact in bacterias:
        bact_seqs = bacterias[bact]
        bact_length = 0
        for seq in bact_seqs:
            bact_length += seq[1] - seq[0]
        match_percent = bact_length / contig_length * 100
        bac_content[bact] = [bact_length, match_percent]
    contig_content[contig] = bac_content

KeyError: 0

In [98]:
most_common_bact = defaultdict(int)
for contig in contig_content:
    bact_content = contig_content[contig]
    full_length
    for bacteria in bact_content:
        length = bact_content[bacteria]
        if length:
            most_common_bact[bacteria] += length[0]

In [110]:
i = 0
for bact in most_common_bact:
    print(bact)
    i+=1
    if i == 10:
        break

Bifidobacterium adolescentis
Bifidobacterium bifidum
Bifidobacterium breve
Bifidobacterium longum
Bifidobacterium kashiwanohense
Bifidobacterium catenulatum;Bifidobacterium
Bos taurus
Clostridium botulinum
Bos mutus
uncultured bacterium


In [56]:
for contig in bact_percentage_in_contig:
    if len(bact_percentage_in_contig[contig]) > 1:
        for bact in bact_percentage_in_contig[contig]:
            lengths = bact_percentage_in_contig[contig][bact]
            print(contig, bact, lengths)

NODE_19_length_67284_cov_62.898968 Bifidobacterium breve [[1, 16826], [16927, 25349], [25450, 39540], [39641, 50368], [50479, 50634], [52155, 53663], [53748, 54342], [54248, 54641], [54937, 59974], [60579, 61652]]
NODE_19_length_67284_cov_62.898968 Bifidobacterium longum [[5873, 6055], [6194, 6365], [6682, 6852], [7250, 7873], [11869, 11999], [18338, 18913], [29854, 39540], [39641, 50368], [50479, 52267], [52498, 52615], [52971, 53663], [53748, 54090], [54091, 54342], [54248, 54641], [54937, 55645], [57299, 57514], [57847, 58805], [60049, 60337], [61500, 61661]]
NODE_25_length_54248_cov_11.563460 Bifidobacterium kashiwanohense [[1, 190], [1471, 2680], [2971, 3889], [8712, 8971], [15561, 15944], [16953, 17730], [19845, 20234], [23833, 24990], [25068, 25646], [26029, 26286], [26297, 27204], [31264, 31308], [32675, 32998], [33220, 33706], [37888, 38242], [46845, 50851]]
NODE_25_length_54248_cov_11.563460 Bifidobacterium longum [[1, 46], [3225, 4276], [21056, 22878], [37869, 37958], [42717

In [21]:
most_common_bact = {}
for contig in bact_percentage_in_contig:
    bacteria_and_length = bact_percentage_in_contig[contig]
    contig_length = int(contig.split("_")[3])
    for bacteria in bacteria_and_length:
        for start_stop in bacteria_and_length[bacteria]:
            length = start_stop[1] - start_stop[0]
            percentage = length / contig_length * 100
            most_common_bact[contig] = [bacteria, ]

NODE_1_length_716629_cov_9.319197 Bifidobacterium adolescentis 196195 27.377485421326796
NODE_2_length_341399_cov_7.303676 Bifidobacterium adolescentis 161492 47.30300908907173
NODE_3_length_159041_cov_54.907126 Bifidobacterium bifidum 64640 40.6436076231915
NODE_4_length_157392_cov_10.870893 Bifidobacterium adolescentis 50651 32.18143234726034
NODE_5_length_150296_cov_63.517637 Bifidobacterium bifidum 41527 27.63014318411668
NODE_6_length_142245_cov_9.064883 Bifidobacterium adolescentis 91331 64.20682625048332
NODE_7_length_139037_cov_8.863516 Bifidobacterium adolescentis 58721 42.234081575407984
NODE_8_length_125379_cov_7.515521 Bifidobacterium adolescentis 44459 35.45968623134656
NODE_9_length_122489_cov_10.121034 Bifidobacterium adolescentis 50988 41.62659504118737
NODE_10_length_107442_cov_70.562587 Bifidobacterium bifidum 38189 35.543828298058486
NODE_11_length_97565_cov_58.932480 Bifidobacterium bifidum 58979 60.45098139701738
NODE_12_length_84447_cov_58.248245 Bifidobacterium b

NODE_705_length_212_cov_7.235294 Bifidobacterium longum 211 99.52830188679245
NODE_705_length_212_cov_7.235294 Bifidobacterium breve 211 99.52830188679245
NODE_706_length_212_cov_5.882353 Bifidobacterium bifidum 211 99.52830188679245
NODE_707_length_212_cov_5.870588 Bifidobacterium bifidum 211 99.52830188679245
NODE_708_length_212_cov_5.411765 Bifidobacterium longum 211 99.52830188679245
NODE_708_length_212_cov_5.411765 Bifidobacterium breve 211 99.52830188679245
NODE_709_length_212_cov_4.470588 Bifidobacterium longum 211 99.52830188679245
NODE_709_length_212_cov_4.470588 Bifidobacterium breve 211 99.52830188679245
NODE_710_length_212_cov_3.400000 Bifidobacterium bifidum 211 99.52830188679245
NODE_711_length_212_cov_3.082353 Bifidobacterium bifidum 211 99.52830188679245
NODE_712_length_212_cov_2.917647 Bifidobacterium bifidum 211 99.52830188679245
NODE_713_length_211_cov_11.428571 Bifidobacterium bifidum 210 99.52606635071089
NODE_714_length_211_cov_10.107143 Bifidobacterium bifidum 21

In [4]:
poopi_poo = {"bacteria" : [[123,500], [750,1230], [3345,7800]]}
print(poopi_poo)
new_poop = [17,250]
for poop in poopi_poo["bacteria"]:
    if new_poop[0] < poop[0] and new_poop[1] > poop[0] and new_poop[1] < poop[1]: 
        poop[0] = new_poop[0]
print(poopi_poo)

{'bacteria': [[123, 500], [750, 1230], [3345, 7800]]}
{'bacteria': [[17, 500], [750, 1230], [3345, 7800]]}


In [95]:
sorted_cont2bact = defaultdict(list)
for contig, value in contig2bactlength.items():
    value.sort(key=lambda x:int(x[0]), reverse=True)
    contig_length = int(contig.split("_")[3])
    contig_bacteria = defaultdict(int)
    for x in value[0:1]:
        bacteria = " ".join(x[1].split()[0:2])
        bacteria_length = int(x[0])
        contig_bacteria[bacteria] += bacteria_length / contig_length * 100
    print(contig, contig_bacteria)
    #print(int(value[0][0])/contig_length * 100)
    #sorted_cont2bact[contig].append(new_value)

NODE_1_length_716629_cov_9.319197 defaultdict(<class 'int'>, {'Bifidobacterium adolescentis': 19.025604601544174})
NODE_2_length_341399_cov_7.303676 defaultdict(<class 'int'>, {'Bifidobacterium adolescentis': 46.50599445223917})
NODE_3_length_159041_cov_54.907126 defaultdict(<class 'int'>, {'Bifidobacterium bifidum': 21.60826453556001})
NODE_4_length_157392_cov_10.870893 defaultdict(<class 'int'>, {'Bifidobacterium adolescentis': 30.327462641049102})
NODE_5_length_150296_cov_63.517637 defaultdict(<class 'int'>, {'Bifidobacterium bifidum': 27.654095917389682})
NODE_6_length_142245_cov_9.064883 defaultdict(<class 'int'>, {'Bifidobacterium adolescentis': 64.25322506942247})
NODE_7_length_139037_cov_8.863516 defaultdict(<class 'int'>, {'Bifidobacterium adolescentis': 42.141300517128535})
NODE_8_length_125379_cov_7.515521 defaultdict(<class 'int'>, {'Bifidobacterium adolescentis': 34.983529937230315})
NODE_9_length_122489_cov_10.121034 defaultdict(<class 'int'>, {'Bifidobacterium adolescent

NODE_273_length_396_cov_1.263941 defaultdict(<class 'int'>, {'Bos mutus': 108.58585858585859})
NODE_274_length_395_cov_0.917910 defaultdict(<class 'int'>, {'Bos mutus': 33.92405063291139})
NODE_275_length_395_cov_0.917910 defaultdict(<class 'int'>, {'Ovis canadensis': 101.0126582278481})
NODE_276_length_395_cov_0.910448 defaultdict(<class 'int'>, {'Bos mutus': 44.81012658227848})
NODE_277_length_395_cov_0.906716 defaultdict(<class 'int'>, {'Bos mutus': 105.31645569620254})
NODE_278_length_395_cov_0.847015 defaultdict(<class 'int'>, {'Ovis canadensis': 100.25316455696202})
NODE_279_length_394_cov_0.707865 defaultdict(<class 'int'>, {'Bos mutus': 100.0})
NODE_280_length_393_cov_0.924812 defaultdict(<class 'int'>, {'Bos mutus': 105.34351145038168})
NODE_281_length_393_cov_0.921053 defaultdict(<class 'int'>, {'Homo sapiens': 100.5089058524173})
NODE_282_length_393_cov_0.917293 defaultdict(<class 'int'>, {'Bos mutus': 100.0})
NODE_283_length_393_cov_0.894737 defaultdict(<class 'int'>, {'Ovi

In [67]:
sorted_cont2bact

defaultdict(list,
            {'NODE_1_length_716629_cov_9.319197': [None],
             'NODE_2_length_341399_cov_7.303676': [None],
             'NODE_3_length_159041_cov_54.907126': [None],
             'NODE_4_length_157392_cov_10.870893': [None],
             'NODE_5_length_150296_cov_63.517637': [None],
             'NODE_6_length_142245_cov_9.064883': [None],
             'NODE_7_length_139037_cov_8.863516': [None],
             'NODE_8_length_125379_cov_7.515521': [None],
             'NODE_9_length_122489_cov_10.121034': [None],
             'NODE_10_length_107442_cov_70.562587': [None],
             'NODE_11_length_97565_cov_58.932480': [None],
             'NODE_12_length_84447_cov_58.248245': [None],
             'NODE_13_length_83012_cov_10.046607': [None],
             'NODE_14_length_82078_cov_6.470049': [None],
             'NODE_15_length_79460_cov_11.216745': [None],
             'NODE_16_length_71672_cov_61.667748': [None],
             'NODE_17_length_68677_cov_68.2

In [40]:
start_in, stop_in = 98543, 234846
start_out, stor_out = 82350, 208359
print(start_in < stop_out <= stop_in)

False
